# 1d dataset for demonstration

Uses the kung dataset:

Females in the Kung dataset

In [1]:
import dp4gp_datasets
import dp4gp
import random
import numpy as np
import GPy
import matplotlib.pyplot as plt
import dp4gp_histogram
import pandas as pd
%matplotlib inline

kung = dp4gp_datasets.load_kung()

X = kung[kung[:,3]==0,2:3]
y = kung[kung[:,3]==0,0:1]
sens = 100.0
epsilon = 1.0
delta = 0.01

middley = (np.max(y)+np.min(y))/2
y[y>middley+sens/2] = middley+sens/2
y[y<middley-sens/2] = middley-sens/2

#ysub = np.mean(y)
ysub = (max(y)+min(y))/2.0
y = y - ysub


ys_std = np.std(y)
y = y / ys_std
ac_sens = sens/ys_std


warning in stationary: failed to import cython module: falling back to numpy


In [9]:
ac_sens

3.8846795501345945

In [55]:
errors = []
testIndices = np.random.choice(range(len(X)),10,replace=False)
Xtest = X[testIndices,:]
ytest = y[testIndices]
Xtrain = np.delete(X,testIndices,0)
ytrain = np.delete(y,testIndices,0)
kern = GPy.kern.RBF(1.0,lengthscale=25.0,variance=1.0)
model = GPy.models.GPRegression(Xtrain,ytrain,kern,normalizer=None)
model.Gaussian_noise = 0.3

dpgp = dp4gp.DPGP_cloaking(model,sens,epsilon,delta)
C = dpgp.get_C(Xtest)
np.max(np.sqrt(np.sum(C**2,0)))

0.029466567343082176

In [67]:
range(0,244,24)

[0, 24, 48, 72, 96, 120, 144, 168, 192, 216, 240]

In [87]:
def runxval(X,y,sens):
    nblocks = 5
    blocksize = len(X)/nblocks #e.g. 244 -> 24 blocksize
    perm = np.random.permutation(len(X))
    block_training_sensitivities = []
    rse = []
    for block in range(0,len(X),blocksize):
        if block+blocksize>=len(X): #if last block to test
            blocksize = len(X)-block
        testIndices = perm[block:(block+blocksize)]
        Xtest = X[testIndices,:]
        ytest = y[testIndices]
        Xtrain = np.delete(X,testIndices,0)
        ytrain = np.delete(y,testIndices,0)

        kern = GPy.kern.RBF(1.0,lengthscale=25.0,variance=1.0)
        model = GPy.models.GPRegression(Xtrain,ytrain,kern,normalizer=None)
        model.Gaussian_noise = 0.3
        
        dpgp = dp4gp.DPGP_cloaking(model,sens,epsilon,delta)
        try:
            samps, mu, samp_cov = dpgp.draw_prediction_samples(Xtest,N=1,Nattempts=1,Nits=150)
        except TypeError:
            continue #we're just not going to be able to include this in the error sum
        C = dpgp.get_C(Xtest)
        block_training_sensitivities.append(np.max(np.sqrt(np.sum((ac_sens * ys_std * C)**2,0))))
        errors = samps-ytest
        rse.append(np.sqrt(np.sum((np.array(errors) * ys_std)**2)))
    print rse
    print block_training_sensitivities
    print ac_sens * ys_std
    
    return rse, block_training_sensitivities, ac_sens*ys_std
rse, trainsens, testsens = runxval(X,y,ac_sens)

(57, 230)
*
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Stopped before convergence
(3.8846795501345945, 3.2552472614374586, array([[ 1.01808662]]), 1.0, 0.0)
(57, 230)
*
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Stopped before convergence
(3.8846795501345945, 3.2552472614374586, array([[ 1.01727203]]), 1.0, 0.0)
(57, 230)
*
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

In [88]:
np.sum(rse)

14.27749003428894

In [90]:
np.sum(np.sort(trainsens)[1:])#+testsens/ blocksize??

2.5072846966499429

 5 blocks => 490.1 | 210.6
10 blocks => 632.9 | 231.6
20 blocks => 963.8 | 232.8
20 blocks => 855.1 | 242.7

In [7]:
normal_sens

(9.331573675206295, 9.331573675206295)